In [211]:
import pandas as pd
import os, fileinput
from tqdm import tqdm

def merge_into_en_kw(df):
    in_pth_base = '../kw/en'
    out_pth_base = 'updated_kws'
    en_0_1_pth = 'kw_en_0_1.txt'
    en_2_3_pth = 'kw_en_2_3.txt'
    en_4_pth = 'kw_en_4.txt'
    en_5_6_7_8_pth = 'kw_en_5_6_7_8.txt'
    en_9_pth = 'kw_en_9.txt'
    # en_all_pth = 'kw_en_keys.txt'
    with open(f'{in_pth_base}/{en_0_1_pth}') as f:
        kws_0_1 = f.readlines()
    with open(f'{in_pth_base}/{en_2_3_pth}') as f:
        kws_2_3 = f.readlines()
    with open(f'{in_pth_base}/{en_4_pth}') as f:
        kws_4 = f.readlines()
    with open(f'{in_pth_base}/{en_5_6_7_8_pth}') as f:
        kws_5_6_7_8 = f.readlines()
    with open(f'{in_pth_base}/{en_9_pth}') as f:
        kws_9 = f.readlines()
    # with open(en_all_pth) as f:
    #     kws_all = f.readlines()

    concept_names = get_en_concept_names()   
        
    merge_candidates = df[df['ICONCLASS'].notnull()]
    merge_candidates = merge_candidates[merge_candidates['ICONCLASS'].str.match('^([0-9])')]
    merge_candidates = merge_candidates[merge_candidates['RELATED SCENT'].notnull()]
    smell_kws = []
    needs_correction = []
    for _, row in tqdm(merge_candidates.iterrows()):
        code = row["ICONCLASS"]
        scents = row["RELATED SCENT"].split(',')
        if code.startswith(('0','1')):
            upd_list = kws_0_1
        elif code.startswith(('2','3')):
            upd_list = kws_2_3
        elif code.startswith('4'):
            upd_list = kws_4
        elif code.startswith(('5','6','7','8')):
            upd_list = kws_5_6_7_8
        elif code.startswith('9'):
            upd_list = kws_9
        else:
            print(row)
        for scent in scents:
            kw_string = f'{row["ICONCLASS"]}|smell:{scent.strip()}{os.linesep}'
            upd_list.append(kw_string)
            # kws_all.append(kw_string)
            if code in concept_names.keys():
                smell_kws.append([code, concept_names[code], scent.strip()])
            else:
                needs_correction.append([code, scent.strip()])
    with open(f'{out_pth_base}/{en_0_1_pth}', 'w') as f:
        f.writelines(sorted(kws_0_1, key=lambda l: (l.split('|')[0], l.split('|')[1])))
    with open(f'{out_pth_base}/{en_2_3_pth}', 'w') as f:
        f.writelines(sorted(kws_2_3, key=lambda l: (l.split('|')[0], l.split('|')[1])))
    with open(f'{out_pth_base}/{en_4_pth}', 'w') as f:
        f.writelines(sorted(kws_4, key=lambda l: (l.split('|')[0], l.split('|')[1])))
    with open(f'{out_pth_base}/{en_5_6_7_8_pth}', 'w') as f:
        f.writelines(sorted(kws_5_6_7_8, key=lambda l: (l.split('|')[0], l.split('|')[1])))
    with open(f'{out_pth_base}/{en_9_pth}', 'w') as f:
        f.writelines(sorted(kws_9, key=lambda l: (l.split('|')[0], l.split('|')[1])))
    # with open(en_all_pth, 'w') as f:
    #     f.writelines(sorted(kws_all, key=lambda l: (l.split('|')[0], l.split('|')[1])))
    return smell_kws, needs_correction

def get_en_concept_names():
    concept_to_name = {}
    base = '../txt/en/txt_en'
    file_pths = [f'{base}_0_1.txt',f'{base}_2_3.txt',f'{base}_4.txt',f'{base}_5_6_7_8.txt',f'{base}_9.txt']
    for pth in file_pths:
        with open(pth) as f:
            for line in f:
                try:
                    concept, name = line.split('|')
                except ValueError:
                    pass
                concept_to_name[concept] = name.rstrip()
    return concept_to_name

In [215]:
get_en_concept_names()['47I53']

'greenhouse, orangery'

In [212]:
smell_kws = []
needs_correction = []
for df_pth in ['odeuropa-taxonomy/spaces.csv', 'odeuropa-taxonomy/objects.csv', 'odeuropa-taxonomy/iconography.csv']:
    df = pd.read_csv(df_pth)
    if 'ICONCLASS' not in df.columns:
        df = df.rename(columns={'ICONCLASS CODE': 'ICONCLASS'})
    if 'RELATED SCENT' not in df.columns:
        df = df.rename(columns={"RELATED SCENT (see 'art historical scent wheel in google drive' for categories)": "RELATED SCENT"})
    smell_kw_per_df, needs_correction_per_df = merge_into_en_kw(df)
    smell_kws.extend(smell_kw_per_df)
    needs_correction.extend(needs_correction_per_df)

2it [00:00, 2726.23it/s]
78it [00:00, 6295.94it/s]
26it [00:00, 2436.26it/s]


In [229]:
check_df = pd.DataFrame(smell_kws).rename(columns={0:'code', 1:'name', 2:'assigned smell'}).to_csv('smell_kws_en.csv', index=None)
# check_df.groupby(['code', 'name'])['assigned smell'].apply(list).to_csv('smell_kws_en.csv', index=None)
pd.DataFrame(needs_correction).to_csv('iconclass_corrections.csv', index=None)
